# EDA of victimas-accionar-represivo-ilegal.csv

## Data ingestion and cleaning

In [1]:
# imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# ingest data and inspect columns

df_raw = pd.read_csv('victimas-accionar-represivo-ilegal.csv')
df_clean = df_raw.copy()

print(df_raw.info())
df_raw.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 15 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   anio_denuncia                               8632 non-null   int64 
 1   tipificacion_ruvte                          8632 non-null   object
 2   id_unico_ruvte                              8632 non-null   object
 3   apellido_paterno_nombres                    8632 non-null   object
 4   apellido_materno                            8330 non-null   object
 5   apellido_casada                             1024 non-null   object
 6   edad_al_momento_del_hecho                   8632 non-null   object
 7   documentos                                  8632 non-null   object
 8   anio_nacimiento                             8632 non-null   object
 9   provincia_pais_nacimiento                   8632 non-null   object
 10  nacionalidad            

,anio_denuncia,tipificacion_ruvte,id_unico_ruvte,apellido_paterno_nombres,apellido_materno,apellido_casada,edad_al_momento_del_hecho,documentos,anio_nacimiento,provincia_pais_nacimiento,nacionalidad,embarazo,fecha_lugar_detencion_secuestro,fecha_lugar_asesinato_o_hallazgo_de_restos,fotografia
0,1984,DESAPARICION FORZADA,ID 5389,ABACHIAN JUAN CARLOS,BEDROSSIAN,NaN,26 años,LE 8293245,1950,BUENOS AIRES,ARGENTINA,NaN,26/12/1976 LA PLATA BUENOS AIRES,---,Sí
1,1984,DESAPARICION FORZADA,ID 87,ABAD ANA CATALINA,SCARLATA,PERUCCA,24 años,LC 10048122,1951,MENDOZA,ARGENTINA,NaN,15/08/1976 CORDOBA CAPITAL CORDOBA,---,Sí
2,1984,DESAPARICION FORZADA,ID 11788,ABAD JULIO RICARDO,CORONEL,NaN,21 años,DNI 10283544,1954,TUCUMAN,ARGENTINA,NaN,NOV/1976 CAPITAL FEDERAL,---,No
3,1984,ASESINATO,ID 9907,ABAD OSCAR GERARDO,DOMATO,NaN,25 años,DNI 10353245,1951,BUENOS AIRES,ARGENTINA,NaN,08/10/1976 LA PLATA BUENOS AIRES,21/10/1976 GRAL. MANSILLA (BARTOLOME BAVIO) ...,No
4,1984,DESAPARICION FORZADA,ID 89,ABAD ROBERTO RODOLFO TOMAS,ZABALA,NaN,23 años,DNI 10650064,1953,CAPITAL FEDERAL,ARGENTINA,NaN,09/08/1976 FLORIDA VICENTE LOPEZ BUENOS AIRES,---,Sí


The following cells clean and process the data as follows:
- `tipificacion_ruvte`: This column describes the case, and stores values such as 'DESAPARICION FORZADA' and 'ASESINATO', often with extra details. I encoded the values into tho following broader categories and added a other/unknown option to handle missing values:
    - 'Child disappearance'
    - 'Forced disappearance'
    - 'Child murder'
    - 'Murder' 
    - 'Negligent death'
    - 'Born in captivity disappearance'
    - 'Other / Unknown'
- `id_unico_ruvte`: This column stores the case ID in the format 'ID xxxx'. I processed the values to be stored as ints. 
- `edad_al_momento_del_hecho`: This column stores the age of the missing person at time of incident in the format 'xx año(s)/mes(es)/dìa(s)'. I processed the values to be stored as ints in years using floor division. 
- `fecha_lugar_detencion_secuestro` and `fecha_lugar_asesinato_o_hallazgo_de_restos`: These columns store the date and location of the dissapearance/murder. I split the date and location into separate columns and parsed the dates into datetime objects. I also encoded ambiguous dates such as 'fines/09/1976' to approximated dates like '28/09/1976'. 



In [3]:
# encode/normalize tipificacion_ruvte

# find unique vals in tipificacion_ruvte
print(f"Unique values in column '{'tipificacion_ruvte'}': {df_raw['tipificacion_ruvte'].unique()} ")

# function to aggregate case typles into broader categories
def normalize_case_type(value):
    val = value.upper()
    if 'DESAPARICION FORZADA' in val:
        if 'NIÑO' in val or 'NIÑA' in val:
            return 'Child disappearance'
        return 'Forced disappearance'
    elif 'ASESINATO' in val:
        if 'NIÑO' in val or 'NIÑA' in val:
            return 'Child murder'
        return 'Murder'
    elif 'NEGLIGENTE' in val:
        return 'Negligent death'
    elif 'NACIDA EN CAUTIVERIO' in val:
        return 'Born in captivity disappearance'
    else:
        return 'Other / Unknown'

df_clean['tipificacion_ruvte'] = df_raw['tipificacion_ruvte'].apply(normalize_case_type).astype('string')

df_clean.head()


Unique values in column 'tipificacion_ruvte': ['DESAPARICION FORZADA' 'ASESINATO'
 'ASESINATO / PROCEDENTE POR APLICACION DE ART. 6º DE LA LEY 24.823'
 'DESAPARICION FORZADA / PROBADO EL DECESO'
 'DESAPARICION FORZADA / EXHUMADOS E IDENTIFICADOS SUS RESTOS'
 'ASESINATO / PROBADO EN CAUSA JUDICIAL'
 'DESAPARICION FORZADA / CON INFORMACION SOBRE EL DECESO (NO PROBADO)'
 'NIÑA "DESAPARECIDA" JUNTO A PADRES O ALLEGADOS'
 'DESAPARICION FORZADA / INVESTIGADO EN CAUSA JUDICIAL'
 'DESAPARICION FORZADA / EN INVESTIGACION'
 'ASESINATO / EXHUMADOS E IDENTIFICADOS SUS RESTOS'
 'NIÑO ASESINADO JUNTO A PADRES O ALLEGADOS'
 'NIÑA ASESINADA JUNTO A PADRES O ALLEGADOS'
 'NIÑO "DESAPARECIDO" JUNTO A PADRES O ALLEGADOS'
 'ASESINATO / PROCEDENTE POR RESOLUCION JUDICIAL'
 'ASESINATO / INVESTIGADO EN CAUSA JUDICIAL'
 'DESAPARICION FORZADA / A DETERMINAR TIPIFICACION'
 'NIÑO FALLECIDO POR TRATO NEGLIGENTE'
 'ASESINATO / PROCEDENTE POR ACTO ADMINISTRATIVO EN APLICACION DE ART. 6º DE LA LEY 24.823'
 'DESAPARIC

,anio_denuncia,tipificacion_ruvte,id_unico_ruvte,apellido_paterno_nombres,apellido_materno,apellido_casada,edad_al_momento_del_hecho,documentos,anio_nacimiento,provincia_pais_nacimiento,nacionalidad,embarazo,fecha_lugar_detencion_secuestro,fecha_lugar_asesinato_o_hallazgo_de_restos,fotografia
0,1984,Forced disappearance,ID 5389,ABACHIAN JUAN CARLOS,BEDROSSIAN,NaN,26 años,LE 8293245,1950,BUENOS AIRES,ARGENTINA,NaN,26/12/1976 LA PLATA BUENOS AIRES,---,Sí
1,1984,Forced disappearance,ID 87,ABAD ANA CATALINA,SCARLATA,PERUCCA,24 años,LC 10048122,1951,MENDOZA,ARGENTINA,NaN,15/08/1976 CORDOBA CAPITAL CORDOBA,---,Sí
2,1984,Forced disappearance,ID 11788,ABAD JULIO RICARDO,CORONEL,NaN,21 años,DNI 10283544,1954,TUCUMAN,ARGENTINA,NaN,NOV/1976 CAPITAL FEDERAL,---,No
3,1984,Murder,ID 9907,ABAD OSCAR GERARDO,DOMATO,NaN,25 años,DNI 10353245,1951,BUENOS AIRES,ARGENTINA,NaN,08/10/1976 LA PLATA BUENOS AIRES,21/10/1976 GRAL. MANSILLA (BARTOLOME BAVIO) ...,No
4,1984,Forced disappearance,ID 89,ABAD ROBERTO RODOLFO TOMAS,ZABALA,NaN,23 años,DNI 10650064,1953,CAPITAL FEDERAL,ARGENTINA,NaN,09/08/1976 FLORIDA VICENTE LOPEZ BUENOS AIRES,---,Sí


In [4]:
# extract id_unico_ruvte as int

# find unique vals in id_unico_ruvte
print(f"Unique values in column '{'id_unico_ruvte'}': {df_raw['id_unico_ruvte'].unique()} ")

# function to extract ID as int
def extract_id(value):
    if 'ID' in value:
        return int(value.split(' ')[1])
    else:
        return pd.NA

df_clean['id_unico_ruvte'] = df_raw['id_unico_ruvte'].apply(extract_id)
df_clean.head() 

Unique values in column 'id_unico_ruvte': ['ID 5389' 'ID 87' 'ID 11788' ... 'ID 10263' 'ID 5384' 'ID 7789'] 


,anio_denuncia,tipificacion_ruvte,id_unico_ruvte,apellido_paterno_nombres,apellido_materno,apellido_casada,edad_al_momento_del_hecho,documentos,anio_nacimiento,provincia_pais_nacimiento,nacionalidad,embarazo,fecha_lugar_detencion_secuestro,fecha_lugar_asesinato_o_hallazgo_de_restos,fotografia
0,1984,Forced disappearance,5389,ABACHIAN JUAN CARLOS,BEDROSSIAN,NaN,26 años,LE 8293245,1950,BUENOS AIRES,ARGENTINA,NaN,26/12/1976 LA PLATA BUENOS AIRES,---,Sí
1,1984,Forced disappearance,87,ABAD ANA CATALINA,SCARLATA,PERUCCA,24 años,LC 10048122,1951,MENDOZA,ARGENTINA,NaN,15/08/1976 CORDOBA CAPITAL CORDOBA,---,Sí
2,1984,Forced disappearance,11788,ABAD JULIO RICARDO,CORONEL,NaN,21 años,DNI 10283544,1954,TUCUMAN,ARGENTINA,NaN,NOV/1976 CAPITAL FEDERAL,---,No
3,1984,Murder,9907,ABAD OSCAR GERARDO,DOMATO,NaN,25 años,DNI 10353245,1951,BUENOS AIRES,ARGENTINA,NaN,08/10/1976 LA PLATA BUENOS AIRES,21/10/1976 GRAL. MANSILLA (BARTOLOME BAVIO) ...,No
4,1984,Forced disappearance,89,ABAD ROBERTO RODOLFO TOMAS,ZABALA,NaN,23 años,DNI 10650064,1953,CAPITAL FEDERAL,ARGENTINA,NaN,09/08/1976 FLORIDA VICENTE LOPEZ BUENOS AIRES,---,Sí


In [5]:
# extract edad_al_momento_del_hecho as int

# find unique vals in edad_al_momento_del_hecho
print(f"Unique values in column '{'edad_al_momento_del_hecho'}': {df_raw['edad_al_momento_del_hecho'].unique()} ")

# function to extract age in years
def extract_age(value):
    if 'año' in value:
        return int(value.split(' ')[0])
    elif 'mes' in value:
        return int(value.split(' ')[0]) // 12
    elif 'día' in value:
        return int(value.split(' ')[0]) // 365
    else:
        return pd.NA

df_clean['edad_al_momento_del_hecho'] = df_raw['edad_al_momento_del_hecho'].apply(extract_age).astype('Int64')
df_clean.head()

Unique values in column 'edad_al_momento_del_hecho': ['26 años' '24 años' '21 años' '25 años' '23 años' '54 años' '27 años'
 '19 años' '33 años' '40 años' '43 años' '29 años' '35 años' '32 años'
 '42 años' '30 años' '17 años' '36 años' '22 años' '50 años' '65 años'
 '20 años' '34 años' '28 años' 'sin datos' '16 años' '39 años' '56 años'
 '48 años' '31 años' '37 años' '18 años' '55 años' '58 años' '38 años'
 '63 años' '1 año' '57 años' '53 años' '5 meses' '15 años' '59 años'
 '51 años' '47 años' '52 años' '13 años' '67 años' '60 años' '41 años'
 '3 años' '5 años' '44 años' '45 años' '49 años' '14 años' '61 años'
 '3 meses' '4 años' '68 años' '64 años' '62 años' '1 mes' '46 años'
 '6 años' '66 años' '74 años' '69 años' '73 años' '1 día' '7 años'
 '77 años' '4 meses' '5 días' '72 años' '9 años' '70 años' '80 años'
 '81 años' '76 años' '7 meses'] 


,anio_denuncia,tipificacion_ruvte,id_unico_ruvte,apellido_paterno_nombres,apellido_materno,apellido_casada,edad_al_momento_del_hecho,documentos,anio_nacimiento,provincia_pais_nacimiento,nacionalidad,embarazo,fecha_lugar_detencion_secuestro,fecha_lugar_asesinato_o_hallazgo_de_restos,fotografia
0,1984,Forced disappearance,5389,ABACHIAN JUAN CARLOS,BEDROSSIAN,NaN,26,LE 8293245,1950,BUENOS AIRES,ARGENTINA,NaN,26/12/1976 LA PLATA BUENOS AIRES,---,Sí
1,1984,Forced disappearance,87,ABAD ANA CATALINA,SCARLATA,PERUCCA,24,LC 10048122,1951,MENDOZA,ARGENTINA,NaN,15/08/1976 CORDOBA CAPITAL CORDOBA,---,Sí
2,1984,Forced disappearance,11788,ABAD JULIO RICARDO,CORONEL,NaN,21,DNI 10283544,1954,TUCUMAN,ARGENTINA,NaN,NOV/1976 CAPITAL FEDERAL,---,No
3,1984,Murder,9907,ABAD OSCAR GERARDO,DOMATO,NaN,25,DNI 10353245,1951,BUENOS AIRES,ARGENTINA,NaN,08/10/1976 LA PLATA BUENOS AIRES,21/10/1976 GRAL. MANSILLA (BARTOLOME BAVIO) ...,No
4,1984,Forced disappearance,89,ABAD ROBERTO RODOLFO TOMAS,ZABALA,NaN,23,DNI 10650064,1953,CAPITAL FEDERAL,ARGENTINA,NaN,09/08/1976 FLORIDA VICENTE LOPEZ BUENOS AIRES,---,Sí


In [6]:
# function to convert mixed date formats to datetime

def parse_mixed_dates(date_str):
    if pd.isna(date_str) or date_str in ['---', '', 'nan', 'sin', 'Sin']:
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    # handle special case: just "2ª" 
    if date_str == '2ª':
        return pd.NaT  # ambiguous, treat as missing
    
    # handle Spanish ordinals (1º = 1st, 2º = 2nd, etc.)
    date_str = date_str.replace('1º', '01').replace('2º', '02').replace('3º', '03')
    date_str = date_str.replace('2ª', '02')  # handle "2ª" -> "02"
    
    # translate Spanish months to English
    spanish_months = {
        'ENE': 'JAN', 'FEB': 'FEB', 'MAR': 'MAR', 'ABR': 'APR',
        'MAY': 'MAY', 'JUN': 'JUN', 'JUL': 'JUL', 'AGO': 'AUG',
        'SEP': 'SEP', 'OCT': 'OCT', 'NOV': 'NOV', 'DIC': 'DEC'
    }
    
    for spanish, english in spanish_months.items():
        date_str = date_str.replace(spanish, english)
    
    # handle Spanish text patterns including "fines" (end of period)
    if 'med' in date_str.lower():
        date_str = date_str.replace('med', '15').replace('MED', '15')
    if 'fin' in date_str.lower() or 'fines' in date_str.lower():
        date_str = date_str.replace('fin', '28').replace('FIN', '28').replace('fines', '28')
    if 'princ' in date_str.lower():
        date_str = date_str.replace('princ', '01').replace('PRINC', '01')
    
    # handle "AÑO/YYYY" and "AÑOS/YYYY-YY" patterns
    if 'AÑO' in date_str.upper():
        if 'AÑOS/' in date_str.upper():
            year_part = date_str.upper().replace('AÑOS/', '')
            if '-' in year_part:
                first_year = year_part.split('-')[0]
            else:
                first_year = year_part
        else:
            first_year = date_str.upper().replace('AÑO/', '')
        
        try:
            return pd.to_datetime(first_year, format='%Y')
        except:
            pass
    
    # handle date ranges with dashes
    if '-' in date_str and '/' in date_str:
        parts = date_str.split('-')
        if len(parts) == 2:
            first_part = parts[0]
            
            # complex day ranges like "31/01-18/02/1977" 
            if date_str.count('/') >= 3:
                if first_part.count('/') == 1:  # "31/01" from "31/01-18/02/1977"
                    year = date_str.split('/')[-1]
                    complete_date = f"{first_part}/{year}"
                    try:
                        return pd.to_datetime(complete_date, format='%d/%m/%Y')
                    except:
                        pass
                elif first_part.count('/') == 2:  # "08/07/1976" from "08/07-31/12/1976"
                    try:
                        return pd.to_datetime(first_part, format='%d/%m/%Y')
                    except:
                        pass
            
            # day ranges like "02-04/11/1977"
            elif date_str.count('/') == 2 and first_part.isdigit():
                month_year = '/'.join(date_str.split('/')[1:])
                reconstructed = f"{first_part}/{month_year}"
                try:
                    return pd.to_datetime(reconstructed, format='%d/%m/%Y')
                except:
                    pass
            
            # month ranges like "FEB-MAR/1977"
            elif date_str.count('/') == 1:
                first_part = first_part + '/' + date_str.split('/')[-1]
                try:
                    return pd.to_datetime(first_part, format='%b/%Y')
                except:
                    pass
            
            # year ranges like "NOV/1975-ABR/1976"
            else:
                try:
                    return pd.to_datetime(first_part, format='%b/%Y')
                except:
                    pass
    
    # try common formats
    formats = ['%d/%m/%Y', '%m/%Y', '%Y', '%b/%Y', '%B/%Y']
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    # fallback: let pandas infer
    try:
        return pd.to_datetime(date_str, dayfirst=True, errors='coerce')
    except:
        return pd.NaT

In [7]:
# split fecha_lugar_detencion_secuestro into date and location and drop original column

df_clean[['fecha_detencion', 'lugar_detencion']] = df_raw['fecha_lugar_detencion_secuestro'].str.split(' ', n=1, expand=True)
df_clean['lugar_detencion'] = df_clean['lugar_detencion'].astype('string')

# clean date columns
df_clean['fecha_detencion'] = df_clean['fecha_detencion'].apply(parse_mixed_dates)
# drop original column
df_clean = df_clean.drop(columns=['fecha_lugar_detencion_secuestro'])

print(f"Date range: {df_clean['fecha_detencion'].min()} to {df_clean['fecha_detencion'].max()}")
print(f"Invalid dates: {df_clean['fecha_detencion'].isna().sum()}")

# diagnose failed date parsing
failed_dates = df_clean[df_clean['fecha_detencion'].isna()]['fecha_detencion'].copy()
print("Sample failed dates:")
print(df_raw.loc[failed_dates.index, 'fecha_lugar_detencion_secuestro'].head(10).tolist())

# check what the original split produced for failed cases
failed_originals = df_raw.loc[failed_dates.index, 'fecha_lugar_detencion_secuestro'].str.split(' ', n=1, expand=True)[0]
print("Failed date strings after split:")
print(failed_originals.value_counts().head(10))

# Check if failures are due to missing/null values vs parsing issues
print(f"\nFailure breakdown:")
print(f"- Null/missing original values: {df_raw.loc[failed_dates.index, 'fecha_lugar_detencion_secuestro'].isna().sum()}")
print(f"- Total parsing failures: {len(failed_dates)}")
#df_clean.head()

Date range: 1962-08-23 00:00:00 to 1983-05-14 00:00:00
Invalid dates: 1132
Sample failed dates:
['---', '---', '---', '---', '---', '---', '---', '---', '---', '---']
Failed date strings after split:
0
---              1076
2ª                  4
fines/1975          4
fines/12/1976       2
fines/10/1976       2
fines/07/1977       2
fines/01/1977       2
sin                 2
fines/06/1976       2
fines/1977          2
Name: count, dtype: int64

Failure breakdown:
- Null/missing original values: 8
- Total parsing failures: 1132


In [8]:
# split fecha_lugar_asesinato_o_hallazgo_de_restos into date and location and drop original column

df_clean[['fecha_asesinato', 'lugar_asesinato']] = df_raw['fecha_lugar_asesinato_o_hallazgo_de_restos'].str.split(' ', n=1, expand=True)
df_clean['lugar_asesinato'] = df_clean['lugar_asesinato'].astype('string')

# clean date columns
df_clean['fecha_asesinato'] = df_clean['fecha_asesinato'].apply(parse_mixed_dates)
# drop original column
df_clean = df_clean.drop(columns=['fecha_lugar_asesinato_o_hallazgo_de_restos'])

print(f"Date range: {df_clean['fecha_asesinato'].min()} to {df_clean['fecha_asesinato'].max()}")
print(f"Invalid dates: {df_clean['fecha_asesinato'].isna().sum()}")

# diagnose failed date parsing
failed_dates = df_clean[df_clean['fecha_asesinato'].isna()]['fecha_asesinato'].copy()
print("Sample failed dates:")
print(df_raw.loc[failed_dates.index, 'fecha_lugar_asesinato_o_hallazgo_de_restos'].head(10).tolist())

# check what the original split produced for failed cases
failed_originals = df_raw.loc[failed_dates.index, 'fecha_lugar_asesinato_o_hallazgo_de_restos'].str.split(' ', n=1, expand=True)[0]
print("Failed date strings after split:")
print(failed_originals.value_counts().head(10))

# Check if failures are due to missing/null values vs parsing issues
print(f"\nFailure breakdown:")
print(f"- Null/missing original values: {df_raw.loc[failed_dates.index, 'fecha_lugar_detencion_secuestro'].isna().sum()}")
print(f"- Total parsing failures: {len(failed_dates)}")


Date range: 1967-01-12 00:00:00 to 1983-05-14 00:00:00
Invalid dates: 6221
Sample failed dates:
['---', '---', '---', '---', '---', '---', '---', '---', nan, '---']
Failed date strings after split:
0
---              6070
sin               139
fines/1975          1
Sin                 1
fines/1978          1
fines/04/1978       1
fines/02/1977       1
fines/08/1978       1
Name: count, dtype: int64

Failure breakdown:
- Null/missing original values: 0
- Total parsing failures: 6221


In [9]:
# normalize 'embarazo' column

def normalize_embarazo(value):
    if pd.isna(value):
        return 'No data'
    v = value.upper()
    if 'PROBABLE' in v or 'APARENTE' in v:
        return 'Probably pregnant'
    elif 'SIN VIDA' in v:
        return 'Pregnant - stillbirth'
    elif 'CAUTIVERIO' in v:
        return 'Pregnant - child born in captivity'
    elif 'RECLUSION' in v:
        return 'Pregnant -  detained'
    elif 'EMBARAZADA' in v:
        return 'Pregnant (confirmed)'
    else:
        return 'No data'

df_clean['embarazo'] = df_raw['embarazo'].apply(normalize_embarazo).astype('string')


In [12]:
# convert other columns to strings/ints/booleans as needed

string_cols = ['apellido_paterno_nombres', 'apellido_materno', 'apellido_casada', 'documentos', 'provincia_pais_nacimiento', 'nacionalidad']

for col in string_cols:
    df_clean[col] = df_raw[col].astype('string')

def convert_to_int(value):
    if value == 'sin datos':
        return pd.NA
    else:
        return int(value)
        
df_clean['anio_nacimiento'] = df_raw['anio_nacimiento'].apply(convert_to_int).astype('Int64')

def convert_to_bool(value):
    if value == 'Sì':
        return True
    elif value == 'No':
        return False
    else:
        return pd.NA

df_clean['fotografia'] = df_raw['fotografia'].map({'Sì': True, 'No': False}).astype('boolean')

In [13]:
print(df_clean.info())
df_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   anio_denuncia              8632 non-null   int64         
 1   tipificacion_ruvte         8632 non-null   string        
 2   id_unico_ruvte             8632 non-null   int64         
 3   apellido_paterno_nombres   8632 non-null   string        
 4   apellido_materno           8330 non-null   string        
 5   apellido_casada            1024 non-null   string        
 6   edad_al_momento_del_hecho  8557 non-null   Int64         
 7   documentos                 8632 non-null   string        
 8   anio_nacimiento            8472 non-null   Int64         
 9   provincia_pais_nacimiento  8632 non-null   string        
 10  nacionalidad               8632 non-null   string        
 11  embarazo                   8632 non-null   string        
 12  fotogr

,anio_denuncia,tipificacion_ruvte,id_unico_ruvte,apellido_paterno_nombres,apellido_materno,apellido_casada,edad_al_momento_del_hecho,documentos,anio_nacimiento,provincia_pais_nacimiento,nacionalidad,embarazo,fotografia,fecha_detencion,lugar_detencion,fecha_asesinato,lugar_asesinato
0,1984,Forced disappearance,5389,ABACHIAN JUAN CARLOS,BEDROSSIAN,<NA>,26,LE 8293245,1950,BUENOS AIRES,ARGENTINA,No data,<NA>,1976-12-26,LA PLATA BUENOS AIRES,NaT,<NA>
1,1984,Forced disappearance,87,ABAD ANA CATALINA,SCARLATA,PERUCCA,24,LC 10048122,1951,MENDOZA,ARGENTINA,No data,<NA>,1976-08-15,CORDOBA CAPITAL CORDOBA,NaT,<NA>
2,1984,Forced disappearance,11788,ABAD JULIO RICARDO,CORONEL,<NA>,21,DNI 10283544,1954,TUCUMAN,ARGENTINA,No data,False,1976-11-01,CAPITAL FEDERAL,NaT,<NA>
3,1984,Murder,9907,ABAD OSCAR GERARDO,DOMATO,<NA>,25,DNI 10353245,1951,BUENOS AIRES,ARGENTINA,No data,False,1976-10-08,LA PLATA BUENOS AIRES,1976-10-21,GRAL. MANSILLA (BARTOLOME BAVIO) MAGDALENA ...
4,1984,Forced disappearance,89,ABAD ROBERTO RODOLFO TOMAS,ZABALA,<NA>,23,DNI 10650064,1953,CAPITAL FEDERAL,ARGENTINA,No data,<NA>,1976-08-09,FLORIDA VICENTE LOPEZ BUENOS AIRES,NaT,<NA>
